In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd

In [2]:
tickers = ['amzn', 'aapl', 'msft', 'goog', 'pypl', 'adbe', 'tsla']
df = yf.download(tickers, start='2015-8-23')

prices = df['Adj Close']
prices.index = pd.to_datetime(prices.index)

mtl_ret = prices.pct_change().resample('M').agg(lambda x: (x+1).prod()-1 if x.isnull().sum()==0 else np.nan)

[*********************100%***********************]  7 of 7 completed


In [8]:
formation = dt.datetime(2016,12,31)
end_measurement = formation - MonthEnd(1)
ret = mtl_ret.loc[end_measurement]
ret = ret.reset_index()

,index,2016-11-30 00:00:00
0,AAPL,-0.021601
1,ADBE,-0.043717
2,AMZN,-0.049695
3,GOOG,-0.033778
4,MSFT,0.012468
5,PYPL,-0.057129
6,TSLA,-0.042128


In [9]:
winners = ret.nlargest(2,end_measurement)
winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners['index'])]
Momentumprofit = winnerret.mean()
winners

,index,2016-11-30 00:00:00
4,MSFT,0.012468
0,AAPL,-0.021601


In [10]:
def momentum(formation):
    end_measurement = formation - MonthEnd(1)
    ret = mtl_ret.loc[end_measurement]
    ret = ret.reset_index()
    winners = ret.nlargest(2,end_measurement)
    winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners['index'])]
    Momentumprofit = winnerret.mean()
    return Momentumprofit

In [31]:
profits = []
dates = []

for i in range(12*6-1):
    profits.append(momentum(formation + MonthEnd(i)))
    dates.append(formation + MonthEnd(i))

dict = {'Profits': profits, 'Date': dates}
profits_df = pd.DataFrame(dict)
profits_df.index = profits_df['Date']
profits_df.drop(['Date'], axis=1, inplace=True)
profits_df.cumsum() * 100

for month in profits:
    


2.0332424412416628

In [12]:
SP = yf.download('^GSPC', start=dates[0],end=dates[-1])['Adj Close']
SP_mtl = SP.pct_change().resample('M').agg(lambda x: (x+1).prod()-1)

[*********************100%***********************]  1 of 1 completed


In [9]:
frame = pd.DataFrame(profits)
frame['SP'] = pd.Series(SP_mtl.values)
frame['excess'] = frame.iloc[:,0] - frame.iloc[:,1]
frame['outperformed'] = ['Yes' if i > 0 else 'No' for i in frame.excess]

In [10]:
cryptos = pd.read_html('https://coinmarketcap.com/all/views/all/')[2]['Symbol'].dropna()

cryptos = [x for x in cryptos if x not in stablecoins]
cryptos

NameError: name 'stablecoins' is not defined

In [11]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import requests
import json

API_KEY = 'AK547FI32J6EWZRH46KY'
SECRET_KEY = 'SR1NeXMzmnNXrKLdFwkxQrkSvfqp04ysb9hb9vSv'

trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)


json_data = requests.get('https://api.alpaca.markets/v2/assets?asset_class=crypto', headers={'Apca-Api-Key-Id':API_KEY, 'Apca-Api-Secret-Key':SECRET_KEY})
results = json_data.json()

out = []
for r in results:
    out.append(r['symbol'])
out


['YFI/USDT',
 'YFI/USD',
 'LINK/BTC',
 'AAVE/USDT',
 'LINK/USD',
 'ALGO/USDT',
 'AVAX/USD',
 'AVAX/USDT',
 'BCH/BTC',
 'BCH/USD',
 'BCH/USDT',
 'BTC/USD',
 'BTC/USDT',
 'WBTC/USD',
 'MATIC/BTC',
 'DAI/USDT',
 'DOGE/USD',
 'DOGE/USDT',
 'ETH/USD',
 'ETH/USDT',
 'LTC/BTC',
 'LTC/USD',
 'LTC/USDT',
 'MATIC/USD',
 'SHIB/USD',
 'USDT/USD',
 'UNI/USDT',
 'UNI/USD',
 'MKR/USD',
 'SOL/USD',
 'SOL/USDT',
 'AAVE/USD',
 'ALGO/USD',
 'TRX/USDT',
 'TRX/USD',
 'BAT/USD',
 'NEAR/USD',
 'NEAR/USDT',
 'SUSHI/USDT',
 'SUSHI/USD',
 'PAXG/USDT',
 'LINK/USDT',
 'PAXG/USD',
 'DAI/USD',
 'GRT/USD',
 'SOL/BTC',
 'UNI/BTC',
 'ETH/BTC']

In [87]:
from binance.client import Client

client  = Client('aWPweK1BwrIX4MbTwHP54hmNI9MqkD1n7AVzuRDW9AqdLi8FmEXrCSVvMrifPhHW','QHGrLOf6caxGzDvq6hsVjTifZ0VeuaUhoeIiom1rbOWenovmniwWFhsMNpyEBDRp', tld='us')
info = client.get_exchange_info()
spot_df = pd.DataFrame(info['symbols'])
spot_df = spot_df[spot_df.quoteAsset == 'USD']
spot_df.index = spot_df.symbol
symbols = spot_df.index.to_list()
# info = pd.DataFrame(client.get_all_tickers())

# info = info['symbol']
# info = [s for s in info if s.endswith('USD')]
# info = [x[:-3] for x in info]
stablecoins = pd.read_html('https://coinmarketcap.com/view/stablecoin/')[0]['Name']
stablecoins = [x[-4:] for x in stablecoins]
def remove(list):
    list = [''.join(x for x in i if x.isalpha() or x.isupper()) for i in list]
             
    return list
stablecoins = remove(stablecoins)
stablecoins = [x + 'USD' for x in stablecoins]
symbols = [x for x in symbols if x not in stablecoins]
results = []
cols = ['symbol', 'price', 'volume']
for s in symbols:
    try:
        klines = client.get_historical_klines(s, Client.KLINE_INTERVAL_1MONTH, start_str='2022-10-01', end_str='2022-10-31')
        for history in klines:
            results.append([s, history[4], history[5]])
    except:
        pass


cryptodf = pd.DataFrame(results, columns=cols)
cryptodf.index = cryptodf['symbol']
cryptodf = cryptodf.drop('symbol', axis=1)
cryptodf = cryptodf.apply(pd.to_numeric)
cryptodf['dollar volume'] = cryptodf['price']*cryptodf['volume']
cryptodf = cryptodf.sort_values('dollar volume', ascending=False)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cryptodf.head(10)


# cryptodf[["volume"]] = cryptodf[["volume"]].apply(pd.to_numeric)
# cryptodf.sort_values(['volume'], ascending=False, inplace=True)


# cryptodf = yf.download(results, start='2022-10-01', end='2022-10-31', ignore_tz=True)
# cryptodf

# info = [x[:-3] for x in info]
# info = [x for x in info if x not in stablecoins]

# result = pd.DataFrame(client.get_historical_klines(info[0], '1d'))
# result

# volumes = pd.DataFrame(client.get_historical_klines('BTCUSD', '1M', '2022-10-01', '2022-10-31'))
# for i in info:
#     volumes = pd.DataFrame()
#     volumes['symbol'] = i
#     volumes.concat(pd.DataFrame(client.get_historical_klines(i, '1M', '2022-10-01', '2022-10-31')))
#     break
# volumes['0'] = volumes['0'].date()
# volumes

,price,volume,dollar volume
symbol,,,
BTCUSD,20490.800,311162.236,6375963142.663
ETHUSD,1572.600,773936.414,1217092403.964
DOGEUSD,0.127,2821245322.000,357451782.297
SOLUSD,32.553,4174025.260,135876209.484
ADAUSD,0.406,284529222.800,115376599.845
MATICUSD,0.903,127367214.300,115012594.513
BNBUSD,325.693,213531.531,69545682.220
LINKUSD,7.869,5970870.180,46984777.446
ATOMUSD,14.336,2959124.711,42422011.857


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cryptodf.head(10)

,symbol,volume
64,SHIBBUSD,325149228368.000
42,DOGEUSD,2821245322.000
86,SLPUSD,1918267897.000
44,VTHOUSD,1252727425.000
145,REEFUSD,1148175847.000
109,JASMYUSD,906394874.100
26,VETUSD,726715909.000
135,ACHUSD,695511093.000
108,SPELLUSD,560218909.000
49,ONEUSD,552468195.200


In [64]:
info = client.get_exchange_info()

spot_df = pd.DataFrame(info['symbols'])
spot_df.columns
# spot_df = spot_df[spot_df.quoteAsset == 'USD']
# spot_df.index = spot_df.symbol
# spot_df.index.to_list()
# spot_df = info[info.permissions == info.permissions[0]]
# spot_df

Index(['symbol', 'status', 'baseAsset', 'baseAssetPrecision', 'quoteAsset',
       'quotePrecision', 'quoteAssetPrecision', 'baseCommissionPrecision',
       'quoteCommissionPrecision', 'orderTypes', 'icebergAllowed',
       'ocoAllowed', 'quoteOrderQtyMarketAllowed', 'allowTrailingStop',
       'cancelReplaceAllowed', 'isSpotTradingAllowed',
       'isMarginTradingAllowed', 'filters', 'permissions'],
      dtype='object')

In [56]:
info

{'timezone': 'UTC',
 'serverTime': 1668132523374,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 100},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 200000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'BTCUSD',
   'status': 'TRADING',
   'baseAsset': 'BTC',
   'baseAssetPrecision': 8,
   'quoteAsset': 'USD',
   'quotePrecision': 4,
   'quoteAssetPrecision': 4,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 2,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelReplaceAllowed': True,
   '

In [89]:
mtl_vol = pd.read_html('https://coinmarketcap.com/currencies/volume/monthly/')
stablecoins = pd.read_html('https://coinmarketcap.com/view/stablecoin/')[0]['Name']
stablecoins = [x[-4:] for x in stablecoins]
def remove(list):
    list = [''.join(x for x in i if x.isalpha() or x.isupper()) for i in list]
             
    return list
stablecoins = remove(stablecoins)
mtl_vol_df = pd.DataFrame(mtl_vol[2])
mtl_vol_df = mtl_vol_df[~mtl_vol_df.Symbol.isin(stablecoins)]

buylist = mtl_vol_df.Symbol.head(11)
buylist


0       BTC
2       ETH
5       BNB
6       SOL
7      DOGE
8       CRO
9     MATIC
10      TRX
11      NEO
12      ADA
13      ONT
Name: Symbol, dtype: object

In [ ]:
import websockets
import json
import pandas as pd

endpoint = 'https://api.binance.com'

test_new_order = '/api/v3/order/test'

msg = json.dumps({
"symbol": "BTCUSDT",
"side": "BUY",
"type": "MARKET",
"quoteOrderQty": 50,
})





